In [1]:
import requests, time
import pandas as pd

f = open("vk.key")
ACCESS_TOKEN = f.read()
f.close()

T = time.localtime() # struct_time
Ts = time.mktime(time.localtime())
#print("Now:", time.strftime("%Y-%m-%d %H:%M:%S", T) )

Tstart = time.strftime("%Y-%m-%d 00:00",time.localtime(Ts - 60*60*24 *(T.tm_wday+7) ))
Tstop  = time.strftime("%Y-%m-%d 00:00",time.localtime(Ts - 60*60*24 * T.tm_wday    ))
#Tstart, Tstop = "2017-01-01 0:00", "2017-06-01 00:00" # ВАРИАНТ ДЛЯ УКАЗАНИЯ ИНТЕРВАЛА ВРУЧНУЮ.
print("Период:", Tstart, '-', Tstop )

fileName = "VKweekly_" + Tstart.split(' ')[0] + '_' + Tstop.split(' ')[0] + '.xlsx'
print("Файл:  ", fileName)

Tstart = time.mktime(time.strptime(Tstart, "%Y-%m-%d %H:%M"))
Tstop = time.mktime(time.strptime(Tstop, "%Y-%m-%d %H:%M"))

Период: 2017-06-19 00:00 - 2017-06-26 00:00
Файл:   VKweekly_2017-06-19_2017-06-26.xlsx


In [4]:
#поиск ID первого сообщения
print("Поиск ID первого сообщения", end='')
P = {"access_token": ACCESS_TOKEN,
         "count": 1, 
         "version":"5.62"
        }
R = requests.get("https://api.vk.com/method/messages.getDialogs?", params = P)

l = 1 # перовое / левое
r = R.json()['response'][1]['mid'] # последнее / правое
m = (r+l)//2 # середина
#print(l, m, r)

while r-l > 1:
    P = {"access_token": ACCESS_TOKEN,
         "message_ids": m,
         "preview_length":0,
        }
    R = requests.get("https://api.vk.com/method/messages.getById?", params = P)
    response = R.json()
    T = response['response'][1]['date']
    #print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(response['response'][1]['date'])))

    if Tstart < T:
        r = m
        m = (l+r)//2
    else:
        l = m
        m = (l+r)//2
        
    #print(l, m, r, '\n')
    time.sleep(0.25)
    print('.', end='')
    
startID = m+1
P = {"access_token": ACCESS_TOKEN,
         "message_ids": startID,
         "preview_length":0,
        }
R = requests.get("https://api.vk.com/method/messages.getById?", params = P)
response = R.json()

print("\nID первого сообщения -", startID)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(response['response'][1]['date'])))




Поиск ID первого сообщения.................
ID первого сообщения - 251295
2017-06-19 00:15:31


In [7]:
# поиск ID последнего сообщения
print("Поиск ID последнего сообщения", end='')
P = {"access_token": ACCESS_TOKEN,
         "count": 1, 
         "version":"5.62"
        }
R = requests.get("https://api.vk.com/method/messages.getDialogs?", params = P)

l = 1 # перовое / левое
r = R.json()['response'][1]['mid'] # последнее / правое
m = (r+l)//2 # середина
#print(l, m, r)

while r-l > 1:
    P = {"access_token": ACCESS_TOKEN,
         "message_ids": m,
         "preview_length":0,
        }
    R = requests.get("https://api.vk.com/method/messages.getById?", params = P)
    response = R.json()
    T = response['response'][1]['date']
    #print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(response['response'][1]['date'])))

    if Tstop < T:
        r = m
        m = (l+r)//2
    else:
        l = m
        m = (l+r)//2
        
    #print(l, m, r, '\n')
    time.sleep(0.25)
    print('.', end='')
    
stopID = m
P = {"access_token": ACCESS_TOKEN,
         "message_ids": stopID,
         "preview_length":0,
        }
R = requests.get("https://api.vk.com/method/messages.getById?", params = P)
response = R.json()

print("\nID последнего сообщения - ", stopID)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(response['response'][1]['date'])))




Поиск ID последнего сообщения..................
ID последнего сообщения -  254497
2017-06-25 23:48:58


In [6]:
response

{'error': {'error_code': 10,
  'error_msg': 'Internal server error: Database problems, try later',
  'request_params': [{'key': 'oauth', 'value': '1'},
   {'key': 'method', 'value': 'messages.getById'},
   {'key': 'preview_length', 'value': '0'},
   {'key': 'message_ids', 'value': '223152'}]}}

In [14]:
# выгружаю сообщения с startID до stoptID и формирую список
print("Формирование списка пользователей", end='')
MESSAGES = list()
strt = startID

while True:
    if stopID - strt > 100:
        message_ids = ','.join( [str(i) for i in range(strt,strt+100)] )
        strt += 100
        
        P = {"access_token": ACCESS_TOKEN,
             "message_ids": message_ids,
             "preview_length":0,
            }
        R = requests.get("https://api.vk.com/method/messages.getById?", params = P)
        MESSAGES.extend( R.json()['response'][1:])
        
        
    else:
        message_ids = ','.join( [str(i) for i in range(strt,stopID+1)] )
        
        P = {"access_token": ACCESS_TOKEN,
             "message_ids": message_ids,
             "preview_length":0,
            }
        R = requests.get("https://api.vk.com/method/messages.getById?", params = P)
        MESSAGES.extend( R.json()['response'][1:])
        print('!')
        break
    time.sleep(1)
    #input()
    print('.', end='')
    
# получили список юзеров
Users = list(frozenset( [u['uid'] for u in MESSAGES ] ))

# сформируем словарь имён юзеров
NAMES = dict()
print("Формирование базы имён", end='')

startU = 0
stopU = len(Users)+1

while stopU - startU > 200:
    user_ids = ','.join( [str(u) for u in Users[startU : startU + 200] ] )
    startU += 200

    P = {"user_ids": user_ids,
         "version":"5.62"
        }
    R = requests.get("https://api.vk.com/method/users.get?", params = P)
    r = R.json()["response"]
    for item in r:
        NAMES[item["uid"]] = item["first_name"] + ' ' + item["last_name"]
    print('.', end='')
    time.sleep(0.25)

user_ids = ','.join( [str(u) for u in  Users[startU : stopU] ] )
P = {"user_ids": user_ids,
     "version":"5.62"
    }
R = requests.get("https://api.vk.com/method/users.get?", params = P)
r = R.json()["response"]
for item in r:
    NAMES[item["uid"]] = item["first_name"] + ' ' + item["last_name"]
print('!')
NAMES[-100045377] = "Центр поддержки Wargaming.net"

Формирование списка пользователей................................!
Формирование базы имён..!


In [16]:
print("""Началась загрузка диалогов. Процесс может занять несколько минут...
('.' - диалог загружен; ',' - диалог превышает 200 сообщений )""")
# формирую столбцы будущей таблицы
Date = list() # 'date'
UserID = list() # 'uid'
Name = list() # Имя юзера
Admin = list() # 'out'
Dialog = list() # Номер диалога
Message = list() # 'body'

# Прохожусь по списку юзеров
i = 1
for user in Users:
    #запрашиваю историю диалога
    offset = 0
    count = 200
    while True:
        P = {"access_token": ACCESS_TOKEN,
             "user_id": user,
             "offset": offset,
             "count": count, # <=200
             "version":"5.62"
            }
        R = requests.get("https://api.vk.com/method/messages.getHistory?", params = P)
        r = R.json()
        time.sleep(0.25)
        
        if len(r['response']) == 1:
            print('?', end='')
            break
        
        for message in r['response'][1:]:
            if startID < message['mid'] < stopID:
                Date.append( time.strftime("%Y-%m-%d %H:%M:%S", time.localtime( message['date'] )) )
                UserID.append( message['uid'] ) 
                try:
                    Name.append( NAMES[message['uid']] )
                except:
                    P = {"user_ids": message['uid'],
                         "version":"5.62"
                        }
                    R = requests.get("https://api.vk.com/method/users.get?", params = P)
                    r2 = R.json()["response"]
                    time.sleep(0.25)
                    for item in r2:
                        NAMES[item["uid"]] = item["first_name"] + ' ' + item["last_name"]
                    Name.append( NAMES[message['uid']] )
                Admin.append( message['out'] )
                Dialog.append( user ) 
                Message.append( message['body'] ) 
            else:
                break
                
        #if r['response'][-1]['mid'] < startID: # 
        if (r['response'][0] - offset)//count == 0: # если в хвосте списка сообщений меннее 200, останавливается
            break
        else: # если нет, делаем новый запрос со сдвигом count
            offset += count
            print(',', end='')
    
    # это просто индикатор прогресса
    if i%100 == 0:
        print('')
    else:
        print('.', end='')
    i += 1
print('!')


Началась загрузка диалогов. Процесс может занять несколько минут...
('.' - диалог загружен; ',' - диалог превышает 200 сообщений )
....................................................................,,...............................
....,.........................................,...........................,...........................
.........................................................,.............................,,,.............
............,,...,.........................,................,......,,....,............,.....................
........................................................!


In [17]:
print("Формирование таблицы данных.")
dataframe = pd.DataFrame( {"Date" : Date, # 'date'
                           "UserID" : UserID, # 'uid'
                           "Name" : Name, # Имя юзера
                           "Admin" : Admin, # 'out'
                           "Dialog" : Dialog, # Номер диалога
                           "Message" : Message # 'body'
                          } )
dataframe["Date"] = dataframe.Date.apply(pd.to_datetime) 
#dataframe.info()
dataframe = dataframe[["Dialog", "Date", "UserID", "Name", "Admin", "Message"]] # перемещение столбцов
dataframe.sort_values(by=["Dialog", "Date"], inplace=True) # сортировка
dataframe.head(10)

print("Запись полученных данных в файл: ", end='')
#удаляем 'https://' и 'http://'
dataframe["Message"] = dataframe["Message"].map(lambda x: x.replace('https://', ''))
dataframe["Message"] = dataframe["Message"].map(lambda x: x.replace('http://', ''))

# Сохраняем полученные данные в файл
dataframe.to_excel(fileName, index=False)
print("готово!")

Формирование таблицы данных.
Запись полученных данных в файл: готово!


In [18]:
# рейтинг админов
df_admin = dataframe[dataframe["Admin"] == 1]["Name"].value_counts()
print("Рейтинг админов:")
print(df_admin)

# рейтинг игроков
df_users = dataframe[dataframe["Admin"] == 0]["Name"].value_counts()
#df_users.head(20)

print("\nCообщений юзеров:", df_users.sum())
print("Уникальных юзеров:", df_users.shape[0])

Рейтинг админов:
Юлия Тараканова    393
Злата Рублева      332
Азур Мягкий        188
Юлия Меньшикова     15
Максим Наумов        6
Name: Name, dtype: int64

Cообщений юзеров: 1672
Уникальных юзеров: 399
